cv2.findChessboardCorners() and cv2.drawChessboardCorners().

1. Use **cv2.findChessboardCorners()** to find corners in chessboard images and aggregate arrays of image points (2D image plane points) and object points (3D world points).
2. Use the OpenCV function **cv2.calibrateCamera()** to compute the calibration matrices and distortion coefficients.
3. Use **cv2.undistort()** to undistort a test image.

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nx = 8
ny = 6
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny * nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./Cal*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)


img = cv2.imread('test_image.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

# Perform undistortion
undist = cv2.undistort(img, mtx, dist, None, mtx)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undist)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## 10. Voxel Grid Downsampling
"pixel" is short for "picture element". Similarly, the word "voxel" is short for "volume element".Each individual cell in the grid is now a voxel and the 3D grid is known as a "voxel grid".A voxel grid filter allows you to **downsample** the data by taking a spatial average of the points in the cloud confined by each voxel.
 Voxel Grid Downsampling section of RANSAC.py.

In [ ]:
# Create a VoxelGrid filter object for our input point cloud
vox = cloud.make_voxel_grid_filter()

# Choose a voxel (also known as leaf) size
# Note: this (1) is a poor choice of leaf size   
# Experiment and find the appropriate size!
LEAF_SIZE = 1   

# Set the voxel (or leaf) size  
vox.set_leaf_size(LEAF_SIZE, LEAF_SIZE, LEAF_SIZE)

# Call the filter function to obtain the resultant downsampled point cloud
cloud_filtered = vox.filter()
filename = 'voxel_downsampled.pcd'
pcl.save(cloud_filtered, filename)

1 LEAF_SIZE implies your voxel is 1 cubic meter in volume. In practice, start with low and ho high.

Ros [image pipeline](http://wiki.ros.org/image_pipeline) For ros calibration you could visit this [link](http://wiki.ros.org/openni_launch/Tutorials/ExtrinsicCalibration)

## 11. Pass Through Filtering

like a cropping tool it allows you to crop any given 3D point cloud by specifying an axis with cut-off values along that axis. The region you allow to pass through, is often referred to as region of interest.

## Random Sample Consensus or "RANSAC
use to identify points in your dataset that belong to a particular model.ie,a plane, a cylinder, a box, or any other common shape.

If you have a prior knowledge of a certain shape being present in a given data set, you can use RANSAC to estimate what pieces of the point cloud set belong to that shape by assuming a particular model.

Remove the tabel using it.

### Disadvantage
no upper limit on the time it can take to compute the model so can use 

fixed number of iterations but that has its own demerits as lower number of iterations, the solution obtained may not be optimal. 

Trade off compute time versus model detection accuracy.

to determine traversable terrain, ground plane segmentation is an important part of a mobile robot’s perception toolkit.



## Ransac
two iteratively repeated steps on a given data set: Hypothesis and Verification. First, a hypothetical shape of the desired model is generated by randomly selecting a minimal subset of n-points and estimating the corresponding shape-model parameters.n=2 for line 3 for ==plane


In [ ]:
# Create the segmentation object
seg = cloud_filtered.make_segmenter()

# Set the model you wish to fit 
seg.set_model_type(pcl.SACMODEL_PLANE)
seg.set_method_type(pcl.SAC_RANSAC)

# Max distance for a point to be considered fitting the model
# Experiment with different values for max_distance 
# for segmenting the table
max_distance = 1
seg.set_distance_threshold(max_distance)

# Call the segment function to obtain set of inlier indices and model coefficients
inliers, coefficients = seg.segment()

## Extracting Indices

In [1]:
# Extract inliers
extracted_inliers = cloud_filtered.extract(inliers, negative=False)
filename = 'extracted_inliers.pcd'
pcl.save(extracted_inliers, filename)

NameError: name 'cloud_filtered' is not defined

 extract method negative parameter to True for removing just the extracted object from the scene

## Outlier removal filter



In [ ]:
# Much like the previous filters, we start by creating a filter object: 
outlier_filter = cloud_filtered.make_statistical_outlier_filter()

# Set the number of neighboring points to analyze for any given point
outlier_filter.set_mean_k(50)

# Set threshold scale factor
x = 1.0

# Any point with a mean distance larger than global (mean distance+x*std_dev) will be considered outlier
outlier_filter.set_std_dev_mul_thresh(x)

# Finally call the filter function for magic
cloud_filtered = outlier_filter.filter()